# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bredasdorp,-34.5322,20.0403,14.46,91,55,1.44,ZA,1681858270
1,1,pingzhuang,42.0372,119.2889,13.01,39,38,3.76,CN,1681858547
2,2,hawaiian paradise park,19.5933,-154.9731,27.73,75,20,8.75,US,1681858547
3,3,ribeira grande,38.5167,-28.7000,14.32,77,71,0.89,PT,1681858548
4,4,bubaque,11.2833,-15.8333,25.82,73,100,8.63,GW,1681858548


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
city_data_df.hvplot.points(
    "Lng"
    ,"Lat"
    ,geo = True
    ,tiles = "OSM"
    ,size='Humidity'
    ,scale = 1.25
    ,frame_width = 800
    ,frame_height = 580
    ,color = "City"
    ,ylabel="Latitide"
    ,xlabel="Longitude"
    ,title="Humidity in Cities"
    ,alpha=0.5
    
)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
# I kept random values that simply returned a larger dataset, as I wasn't finding any hotels below...
nice_df = city_data_df.loc[(city_data_df['Max Temp'] <= 30)
                           & (city_data_df['Humidity'] > 5)
                           & (city_data_df['Wind Speed'] > 1)]
# Drop any rows with null values
nice_df = nice_df.dropna(how='any')

#Display sample data
nice_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bredasdorp,-34.5322,20.0403,14.46,91,55,1.44,ZA,1681858270
1,1,pingzhuang,42.0372,119.2889,13.01,39,38,3.76,CN,1681858547
2,2,hawaiian paradise park,19.5933,-154.9731,27.73,75,20,8.75,US,1681858547
4,4,bubaque,11.2833,-15.8333,25.82,73,100,8.63,GW,1681858548
5,5,bethel,41.3712,-73.4140,11.38,50,0,5.66,US,1681858548
...,...,...,...,...,...,...,...,...,...,...
551,551,mili,6.0815,171.7350,27.74,77,100,7.21,MH,1681858658
552,552,aden,12.7794,45.0367,28.81,71,1,8.64,YE,1681858658
553,553,kosh-agach,50.0000,88.6667,-18.68,82,26,1.52,RU,1681858658
554,554,dryden,49.7833,-92.7503,6.16,22,0,3.60,CA,1681858659


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = nice_df.loc[:, ['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

hotel_df['Lat'] = hotel_df['Lat'].astype(float)
hotel_df['Lng'] = hotel_df['Lng'].astype(float)

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,bredasdorp,ZA,-34.5322,20.0403,91,
1,pingzhuang,CN,42.0372,119.2889,39,
2,hawaiian paradise park,US,19.5933,-154.9731,75,
4,bubaque,GW,11.2833,-15.8333,73,
5,bethel,US,41.3712,-73.4140,50,
...,...,...,...,...,...,...
551,mili,MH,6.0815,171.7350,77,
552,aden,YE,12.7794,45.0367,71,
553,kosh-agach,RU,50.0000,88.6667,82,
554,dryden,CA,49.7833,-92.7503,22,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
# I believe that the code below is correct, but I am not getting very many results, but I'm not seeing any errors.

radius = 10000
params = {'categories': 'accommodation.hotel'
    ,"limit": 1
    ,"apiKey": geoapify_key
    ,"lang":"en"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude, longitude = row['Lat'], row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{latitude},{longitude},{radius}'
    params["bias"] = f'proximity:{latitude},{longitude}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
hotel_df

Starting hotel search
bredasdorp - nearest hotel: No hotel found
pingzhuang - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
bubaque - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
iqaluit - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
olonkinbyen - nearest hotel: No hotel found
west island - nearest hotel: No hotel found
pimentel - nearest hotel: No hotel found
broome - nearest hotel: No hotel found
sault ste. marie - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
shengli - nearest hotel: No hotel found
saint-philippe - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
solnechnyy - nearest hotel: No hotel found
kapuskasing - nearest hotel: No hotel found
utrik - nearest hotel: No hotel found
ad dabbah - nearest hotel: No hotel found
georgetown - nearest hotel: No hotel found
uturoa - nearest hotel: No hotel found
whitehorse - nearest hot

shingu - nearest hotel: No hotel found
port elizabeth - nearest hotel: No hotel found
dongola - nearest hotel: No hotel found
rechka-vydrino - nearest hotel: No hotel found
lerwick - nearest hotel: No hotel found
gisborne - nearest hotel: No hotel found
fort-shevchenko - nearest hotel: No hotel found
dunedin - nearest hotel: No hotel found
santa ana - nearest hotel: No hotel found
scone - nearest hotel: No hotel found
sompeta - nearest hotel: No hotel found
susuman - nearest hotel: No hotel found
vorgashor - nearest hotel: No hotel found
tommot - nearest hotel: No hotel found
san antonio de pale - nearest hotel: No hotel found
seymour - nearest hotel: No hotel found
port saint john's - nearest hotel: No hotel found
al kut - nearest hotel: No hotel found
burutu - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
tazovsky - nearest hotel: No hotel found
flin flon - nearest hotel: No hotel found
bandarbeyla - nearest hotel: Pizzaria "Ararat"
ngaoundere - nearest hotel: No

val-des-monts - nearest hotel: No hotel found
keelung - nearest hotel: No hotel found
idaho falls - nearest hotel: No hotel found
charters towers - nearest hotel: No hotel found
mellouleche - nearest hotel: No hotel found
lorengau - nearest hotel: No hotel found
brookings - nearest hotel: No hotel found
ulsteinvik - nearest hotel: No hotel found
liman - nearest hotel: خانه معلم
sidhi - nearest hotel: No hotel found
port-vila - nearest hotel: No hotel found
malvan - nearest hotel: No hotel found
morehead city - nearest hotel: No hotel found
zaqatala - nearest hotel: No hotel found
severo-vostotchnyi bank - nearest hotel: No hotel found
brisas de zicatela - nearest hotel: No hotel found
melfi - nearest hotel: No hotel found
edna - nearest hotel: No hotel found
eraan - nearest hotel: No hotel found
talitsa - nearest hotel: No hotel found
milton - nearest hotel: No hotel found
kone - nearest hotel: No hotel found
thinadhoo - nearest hotel: No hotel found
antonio enes - nearest hotel: No ho

,City,Country,Lat,Lng,Humidity,Hotel Name
0,bredasdorp,ZA,-34.5322,20.0403,91,No hotel found
1,pingzhuang,CN,42.0372,119.2889,39,No hotel found
2,hawaiian paradise park,US,19.5933,-154.9731,75,No hotel found
4,bubaque,GW,11.2833,-15.8333,73,No hotel found
5,bethel,US,41.3712,-73.4140,50,No hotel found
...,...,...,...,...,...,...
551,mili,MH,6.0815,171.7350,77,No hotel found
552,aden,YE,12.7794,45.0367,71,No hotel found
553,kosh-agach,RU,50.0000,88.6667,82,No hotel found
554,dryden,CA,49.7833,-92.7503,22,No hotel found


In [11]:
%%capture --no-display
#drop the rows without hotels found
hotel_df = hotel_df[hotel_df['Hotel Name'] != "No hotel found"]

# Configure the map plot, plotting 'cities' where we found hotels
hotel_df.hvplot.points(
    "Lng"
    ,"Lat"
    ,geo = True
    ,tiles = "OSM"
    ,size='Humidity'
    ,scale = 1.25
    ,frame_width = 800
    ,frame_height = 580
    ,color = "City"
    ,ylabel="Latitide"
    ,xlabel="Longitude"
    ,alpha=0.5
    ,hover_cols=['City', 'Humidity', 'Hotel Name', 'Country']
)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)